In [6]:
#imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [7]:
# create an app name
spark = SparkSession.builder.appName("insta-cart-project").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/03 04:27:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [20]:
#(Task 1 step 1) Read the orders data as a dataframe in pySpark 
orders =spark.read.csv("insta-cart/insta-cart/orders.csv", header = True, inferSchema = True)

In [22]:
#(Task 1 step 2) Display the data up to 10 rows
orders.show(10)

+--------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
| 2539329|      1|   prior|           1|        2|                8|                  NULL|
| 2398795|      1|   prior|           2|        3|                7|                  15.0|
|  473747|      1|   prior|           3|        3|               12|                  21.0|
| 2254736|      1|   prior|           4|        4|                7|                  29.0|
|  431534|      1|   prior|           5|        4|               15|                  28.0|
| 3367565|      1|   prior|           6|        2|                7|                  19.0|
|  550135|      1|   prior|           7|        1|                9|                  20.0|
| 3108588|      1|   prior|           8|        1|               14|            

In [26]:
#(Task 2 step 1) replacing NULL values with 99 o column days_since_prior_order
Task2_step1_result = orders.na.fill(value=999, subset=["days_since_prior_order"]) 

In [27]:
# (Task 2 step 2) Display the data up to 10 rows
Task2_step1_result.show(10)

+--------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
| 2539329|      1|   prior|           1|        2|                8|                 999.0|
| 2398795|      1|   prior|           2|        3|                7|                  15.0|
|  473747|      1|   prior|           3|        3|               12|                  21.0|
| 2254736|      1|   prior|           4|        4|                7|                  29.0|
|  431534|      1|   prior|           5|        4|               15|                  28.0|
| 3367565|      1|   prior|           6|        2|                7|                  19.0|
|  550135|      1|   prior|           7|        1|                9|                  20.0|
| 3108588|      1|   prior|           8|        1|               14|            

In [28]:
# Registering your PySpark DataFrame as a temporary SQL table inside your Spark session
orders.createOrReplaceTempView("orders")

In [29]:
# Display the result that contains the total orders placed on each day of the week (Monday to Sunday) 
Task3_results = spark.sql("""
SELECT COUNT(order_id) AS total_orders, (
CASE
    WHEN order_dow = '0' THEN 'Sunday'
    WHEN order_dow = '1' THEN 'Monday'
    WHEN order_dow = '2' THEN 'Tuesday'
    WHEN order_dow = '3' THEN 'Wednesday'
    WHEN order_dow = '4' THEN 'Thursday'
    WHEN order_dow = '5' THEN 'Friday'
    WHEN order_dow = '6' THEN 'Saturday'
END
) AS day_of_week
FROM orders
GROUP BY order_dow
ORDER BY total_orders DESC
                                   
""").show()

+------------+-----------+
|total_orders|day_of_week|
+------------+-----------+
|      600905|     Sunday|
|      587478|     Monday|
|      467260|    Tuesday|
|      453368|     Friday|
|      448761|   Saturday|
|      436972|  Wednesday|
|      426339|   Thursday|
+------------+-----------+



In [30]:
# Task 4 - Select the number of order IDs as Total_Orders and the hour at which the order was placed using spark sql
Task4_result = spark.sql("""
SELECT 
    COUNT(order_id) AS total_orders, 
    order_hour_of_day AS hour 
FROM Orders
GROUP BY order_hour_of_day
ORDER BY order_hour_of_day
""")

In [31]:
Task4_result.show() #Display the result that contains total orders and the hour

+------------+----+
|total_orders|hour|
+------------+----+
|       22758|   0|
|       12398|   1|
|        7539|   2|
|        5474|   3|
|        5527|   4|
|        9569|   5|
|       30529|   6|
|       91868|   7|
|      178201|   8|
|      257812|   9|
|      288418|  10|
|      284728|  11|
|      272841|  12|
|      277999|  13|
|      283042|  14|
|      283639|  15|
|      272553|  16|
|      228795|  17|
|      182912|  18|
|      140569|  19|
+------------+----+
only showing top 20 rows



In [38]:
## task 5 step 1 Read the order_products__prior.csv and products.csv data
products = spark.read.csv("insta-cart/insta-cart/products.csv", header = True, inferSchema = True)
order_product =spark.read.csv("insta-cart/insta-cart/order_products__prior.csv", header = True, inferSchema = True)

In [40]:
# Store the tables for a particular spark session 
products.createOrReplaceTempView("products")
order_product.createOrReplaceTempView("order_product")

In [44]:
task5_result=spark.sql("""
SELECT 
    COUNT(opp.order_id) AS Order_Count,
    p.product_name AS Popular_Product_Name
FROM order_product opp
JOIN products p
ON opp.product_id = p.product_id
GROUP BY p.product_name
ORDER BY order_count DESC
LIMIT 10
""")

In [45]:
# Display the result
task5_result.show()

+-----------+--------------------+
|Order_Count|Popular_Product_Name|
+-----------+--------------------+
|     472565|              Banana|
|     379450|Bag of Organic Ba...|
|     264683|Organic Strawberries|
|     241921|Organic Baby Spinach|
|     213584|Organic Hass Avocado|
|     176815|     Organic Avocado|
|     152657|         Large Lemon|
|     142951|        Strawberries|
|     140627|               Limes|
|     137905|  Organic Whole Milk|
+-----------+--------------------+



In [52]:
## task 6 & 7 Read the data from the departements.csv file 
departments = spark.read.csv("insta-cart/insta-cart/departments.csv", header = True, inferSchema = True)
# Store the tables for a particular spark session 
departments.createOrReplaceTempView("departments")
task6n7_result=spark.sql("""
SELECT 
        department_id AS Department_ID, 
        COUNT(product_id) AS Max_Product
    FROM products
    GROUP BY department_id
    ORDER BY Max_Product DESC
    LIMIT 1

""")

In [53]:
#Display the data stored 
task6n7_result.show()

+-------------+-----------+
|Department_ID|Max_Product|
+-------------+-----------+
|           11|       6563|
+-------------+-----------+

